# Graph Neural Networks for Decentralized Multi-Robot Path Planning In Warehouse

### Importing the libraries

In [1]:
from Dataset_Generator import DatasetGenerator 
import numpy as np
import matplotlib.pyplot as plt
import torch

### Part 1 - Dataset Generation

In [2]:
grid = [[0 for _ in range(20)] for _ in range(20)]  # Generate grid map
dataset_generator = DatasetGenerator(num_cases=5000, num_agents=6, grid=grid)
# cases = dataset_generator.generate_cases()
# dataset_generator.save_cases_to_file(cases, "dataset.json")
# print(f"Generated and saved {len(cases)} cases.")
cases = dataset_generator.load_cases_from_file("dataset.json")

### Part 2 - Pre-Processing

In [3]:
from pre_processing import Preprocessing
p = Preprocessing(grid,cases,3)
data_tensors  = p.begin()

In [4]:
# with open("data_tensors.txt", 'w') as file:
#     file.write(str(data_tensors))

### Part 3 - Encoding

In [5]:
from Encoder import Encode
encoder = Encode(data_tensors,6)
encoded_tensors = encoder.begin()

In [6]:
# with open("encoded_tensors.txt", 'w') as file:
#     file.write(str(encoded_tensors))

### Part 4 - GNN

##### Get Adjacency Matrix

In [7]:
from Adjacency_Matrix import adj_mat
adj = adj_mat(cases,3)
adj_matrices = adj.get_adj_mat()

In [8]:
# with open("Adj_Matrices.txt", 'w') as file:
#     file.write(str(adj_matrices))

##### Create GNN Model

In [ ]:
from GNN_file import Communication_GNN
comm_gnn = Communication_GNN(encoded=encoded_tensors, adj_mat=adj_matrices, num_of_agents=6)
gnn_features = comm_gnn.begin()

c:\Users\moham\anaconda3\Lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


In [11]:
# with open("GNN_Features.txt", 'w') as file:
#     file.write(str(gnn_features))

##### Extract Actions

In [12]:
from Extarct_Actions import Action_Extractor
action_extractor = Action_Extractor(cases, 6)
actions = action_extractor.extract()

In [13]:
gnn_features = torch.cat([gnn_features[key][subkey] for key in gnn_features for subkey in gnn_features[key]])

In [14]:
actions = torch.tensor([actions[key][subkey] for key in actions for subkey in actions[key]])

In [15]:
reshaped_gnn_features = []
temp = []
c = 1
for i in range(len(gnn_features)):
    temp.append(gnn_features[i].tolist())
    if c % 6 == 0:
        reshaped_gnn_features.append(torch.Tensor(temp))
        temp = []
    c+=1

In [16]:
dataset = []
for i in range(len(actions)):
    dataset.append((reshaped_gnn_features[i],actions[i]))

In [ ]:
from MLP_Action import ActionMLP
mlp = ActionMLP(input_dim= 128, hidden_dim= 128, num_actions= 5)

In [ ]:
from Model_Generation import Generate_Model
model = Generate_Model(model = mlp, dataset=dataset,num_epochs=300)

In [ ]:
model.train_model()

Epoch [1/300] - Train Loss: 1.1662 - LR: 0.001000 - Val Loss: 1.1284 - Val Acc: 77.30%
Epoch [2/300] - Train Loss: 1.1331 - LR: 0.001000 - Val Loss: 1.1214 - Val Acc: 77.97%
Epoch [3/300] - Train Loss: 1.1264 - LR: 0.001000 - Val Loss: 1.1179 - Val Acc: 78.35%
Epoch [4/300] - Train Loss: 1.1232 - LR: 0.001000 - Val Loss: 1.1131 - Val Acc: 78.78%
Epoch [5/300] - Train Loss: 1.1205 - LR: 0.001000 - Val Loss: 1.1103 - Val Acc: 79.07%
Epoch [6/300] - Train Loss: 1.1185 - LR: 0.001000 - Val Loss: 1.1086 - Val Acc: 79.25%
Epoch [7/300] - Train Loss: 1.1164 - LR: 0.001000 - Val Loss: 1.1082 - Val Acc: 79.28%
Epoch [8/300] - Train Loss: 1.1152 - LR: 0.001000 - Val Loss: 1.1052 - Val Acc: 79.55%
Epoch [9/300] - Train Loss: 1.1137 - LR: 0.001000 - Val Loss: 1.1059 - Val Acc: 79.53%
Epoch [10/300] - Train Loss: 1.1128 - LR: 0.001000 - Val Loss: 1.1045 - Val Acc: 79.64%
Epoch [11/300] - Train Loss: 1.1118 - LR: 0.001000 - Val Loss: 1.1039 - Val Acc: 79.72%
Epoch [12/300] - Train Loss: 1.1111 - LR:

In [ ]:
model.test_model()

Test Loss: 1.0915, Test Accuracy: 80.94%


(1.0914835617667258, 0.8094084626179903)